In [20]:
import os, sys
sys.path.append("..")

import torch

from transformers import AutoTokenizer, GPTNeoXForCausalLM as HF_GPTNeoXForCausalLM
from peft_pretraining.modeling_gpt_neox import GPTNeoXForCausalLM
from optimum.bettertransformer import BetterTransformer

In [21]:
model_name = "EleutherAI/pythia-1.4b"
orig_model = BetterTransformer.transform(HF_GPTNeoXForCausalLM.from_pretrained(model_name))

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
input_ids = tokenizer("Hello, my dog is cute", return_tensors="pt").input_ids

In [6]:
orig_out = orig_model(input_ids)

In [5]:
flashattention_model = GPTNeoXForCausalLM.from_pretrained(model_name)

In [7]:
flashattention_out = flashattention_model(input_ids)

In [8]:
orig_out.logits

tensor([[[ 4.9625, -4.6787,  9.8467,  ..., -4.7095, -4.6852, -4.9827],
         [ 3.3209, -4.9020,  4.0805,  ..., -5.0333, -4.8204, -4.9142],
         [ 0.1438, -4.6176,  4.7885,  ..., -4.5550, -4.4848, -4.6801],
         [ 5.7205, -3.3621, 12.3057,  ..., -3.1542, -3.3429, -3.3110],
         [ 2.7800, -3.1916,  4.4636,  ..., -2.9114, -3.3944, -3.2033],
         [ 8.5833, -2.5008, 13.7838,  ..., -2.5503, -2.4390, -2.5776]]],
       grad_fn=<UnsafeViewBackward0>)

In [9]:
flashattention_out.logits

tensor([[[ 4.9625, -4.6787,  9.8467,  ..., -4.7095, -4.6852, -4.9827],
         [ 3.3209, -4.9020,  4.0805,  ..., -5.0333, -4.8204, -4.9142],
         [ 0.1438, -4.6176,  4.7884,  ..., -4.5550, -4.4848, -4.6801],
         [ 5.7205, -3.3621, 12.3057,  ..., -3.1542, -3.3429, -3.3110],
         [ 2.7800, -3.1916,  4.4636,  ..., -2.9114, -3.3944, -3.2033],
         [ 8.5833, -2.5008, 13.7838,  ..., -2.5503, -2.4390, -2.5776]]],
       grad_fn=<UnsafeViewBackward0>)

In [12]:
torch.allclose(orig_out.logits, flashattention_out.logits, atol=1e-4)

False

In [22]:
out_gen = orig_model.generate(input_ids)
tokenizer.decode(out_gen[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/mnt/shared_home/vlialin/miniconda3/envs/peft_pretraining_shala/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Hello, my dog is cute, but he\'s not a dog. He\'s a cat."\n'

In [18]:
out_gen = flashattention_model.generate(input_ids)
tokenizer.decode(out_gen[0].tolist())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


'Hello, my dog is cute, I am. I am. I am. I am. I'